In [1]:
from process_outputs import *
import glob
from utils import get_reg_ids, checkfile
import os

reg_ids = pd.read_csv('../databases/biorisk/reg_taxids', header=None)

try:
    reg_ids
except:
    reg_ids = get_reg_ids()

# query_files = glob.glob('blast/*.trembl.blastx.pool')
# query_files = glob.glob('blast/NCBI*.sprot.blastx')

# queries = []
# for file in query_files:
#     if checkfile(file):
#         query = file.replace('.fasta', '')
#         query = file.replace('blast/', '')
#         query = query.replace('.nr.blastn', '')
#         query = query.replace('.nr.blastx.pool', '')
#         query = query.replace('.sprot.blastx', '')
#         queries.append(query)

queries = ["microarray_sample", "gfp", "b.anthracis_dnaA", "b.anthracis_lef", "f.tularensis_pillin", "b.anthracis_lef_ecoli_optimized", "b.anthracis_lef_frameshift", "b.anthracis_dnaA_spikedwith_lef", "VFDB_setA_nt_1115"]
# queries = ["f.tularensis_pillin"]

for query in queries:
    print(query)
#     blast = plot_tax(query, reg_ids, "nt.blastn.pool")
    blast = plot_tax(query, reg_ids, "nr.blastx.pool")
#     sprot(query, 5)
#     biorisk(query, 5)
#     vfdb(query, 5)
#     img = results_page(query)
#     img.save("summaries/" + query + ".png")
#       vfdb(query, 5)
#     try:
#         plot_tax(query, reg_ids, "nr.blastx.pool")
        
#     if os.path.isfile('figures/' + query + '_sprotfunsocs.png') & os.path.isfile('figures/' + query + '_tax_pie.png'):
#         print(query)
#         plot_tax(query, reg_ids, "nr.blastn")
#         biorisk(query, 5)
        
#     except:
#         'nevermind'


microarray_sample
Taxon id 2796383 is missing
Taxon id 2796391 is missing
Taxon id 2803857 is missing
Taxon id 2796377 is missing
Taxon id 2796378 is missing
Taxon id 2796388 is missing
Taxon id 2796389 is missing
Taxon id 2796385 is missing
gfp
Taxon id 2767773 is missing
Taxon id 2798852 is missing
Taxon id 2813847 is missing
Taxon id 2813848 is missing
Taxon id 2813849 is missing
Taxon id 2703893 is missing
b.anthracis_dnaA
Taxon id 2810349 is missing
Taxon id 2809036 is missing
Taxon id 2804502 is missing
Taxon id 2803857 is missing
Taxon id 2796379 is missing
Taxon id 2796378 is missing
Taxon id 2796383 is missing
Taxon id 2796385 is missing
Taxon id 2796388 is missing
Taxon id 2796389 is missing
Taxon id 2796391 is missing
Taxon id 2796395 is missing
Taxon id 2796408 is missing
Taxon id 2796409 is missing
Taxon id 2796414 is missing
b.anthracis_lef
f.tularensis_pillin
Taxon id 2803918 is missing
b.anthracis_lef_ecoli_optimized
b.anthracis_lef_frameshift
b.anthracis_dnaA_spikedwit

In [13]:
blast = plot_tax("igem_parts", reg_ids, "nr00.blastx")

KeyboardInterrupt: 

In [1]:
# Swissprot annotations

from process_outputs import *
from utils import *

def sprot(query, nhits):
    file = 'blast/' + query + ".sprot.blastx"
    if checkfile(file) == 0:
        return
    blast = readblast(file)
#     blast = trimblast(blast)
    blast.reset_index(drop=True)
    blast = blast.iloc[0:nhits,:]
    names = []
    for x in blast['subject title']:
        if (re.search(r"RecName: Full=.*(;|\[)", x)) != None:
            result = re.search(r"RecName: Full=.*(;|\[)", x)
            name = result.group(0)
            name = re.sub("\[", "", name)
            name = re.sub("RecName: Full=", "", name)
            name = re.sub("; Short.*", "", name)
            names.append(name)
        else:
            print("Name error: ", x)
    
    blast['flag'] = ''
    funsocs = []
    goterms = []
    rel_keys = []
    oth_keys = []
    
    nhits = blast.shape[0]+1
    
    for i in range(0, blast.shape[0]):
        sprotid = blast.iloc[i,2]
        handle = ExPASy.get_sprot_raw(sprotid)
        record = SwissProt.read(handle)
        rel_keys = add_rel_keys(rel_keys, record)
        oth_keys = add_keys(oth_keys, record)
        if rel_keys[i] != '':
            blast.loc[i,'flag'] = True
        else:
            blast.loc[i,'flag'] = False

    desc_format = "{:<8} {:<20} {:<20}"
    descriptions = []
    for i in range(blast.shape[0]):
        sep = ' | '
        descriptions.append(sep.join([blast.iloc[i,2], rel_keys[i], names[i], oth_keys[i]]))
    if len(descriptions) < blast.shape[0]:
        print('Error - length of descriptions does not match length of hits', descriptions)
    
#     print(descriptions)
    
#     fig = plothits(blast['q. start'], blast['q. end'], blast['query length'][0], descriptions, pd.to_numeric(blast['% identity']), nhits)
#     fig.update_layout(showlegend=False, title={'text': 'Swissprot hits and relevant functional annotations', 'y':1, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'})
#     fig.write_image("figures/" + query + "_sprotfunsocs.png", width=1000, height=60*nhits+60, scale=2)
    
    return blast, descriptions

def plothits(starts, ends, qlen, names, colours, nhits):
    fig = go.Figure(go.Scatter(x=[0,0], y=[0,0], mode="markers",marker=dict(color=[80,100], colorscale="YlOrRd",colorbar=dict(title="% identity", x=-0.15, xanchor="left"))), go.Layout(plot_bgcolor="white"))
    fig.add_shape(type="rect", name = 'Query', x0 = 1, x1 = qlen, y0=0.5, y1=1.3, line=dict(color="white"), fillcolor='grey')
    fig.add_annotation(xanchor='left', text='Query', x=qlen, y=(0.5+1.3)/2,font=dict(family="Arial Narrow", size=int(100/nhits), color="#000000"), bgcolor="#ffffff", showarrow=False) # Courier New, monospace
    for i in range(0, starts.shape[0]):
        start = starts[i]
        end = ends[i]
        hit_description = names[i]
        colour = colours[i]
        fig.add_shape(type="rect", name = hit_description, x0 = start, x1 = end, y0=0.5+i+1, y1=1.3+i+1, line=dict(color="white"), fillcolor=matplotlib.colors.rgb2hex(cm.YlOrRd(colour)[:3]))
        fig.add_trace(go.Scatter(text = str(colour), x = [start, end, end, start, start], y=[0.5+i+1, 0.5+i+1, 1.3+i+1, 1.3+i+1, 0.5+i+1], fill="toself", line=dict(color="white")))
        fig.add_annotation(xanchor='left', text=hit_description, x=qlen, y=(0.5+i+1+1.3+i+1)/2,font=dict(family="Arial Narrow", size=int(100/nhits), color="#000000"), bgcolor="#ffffff", showarrow=False) # Courier New, monospace
    fig.update_xaxes(range=[0, qlen*2+3000])
    fig.update_yaxes(range=[nhits+0.5, 0.5], showticklabels=False)
    fig.update_layout(margin=dict(l=10, r=20, t=30, b=0))
    fig.clim(50,100)
    return fig

query = 'accA1'
suffix = '.nr.blastx.pool'
[blast, descriptions] = sprot(query, 20)
print("done")
blast = readblast(file)

done


In [23]:
handle = ExPASy.get_sprot_raw('Q9I750')
record = SwissProt.read(handle)
record.keywords
record.seqinfo

(344, 36679, '3321BAC4C8DB025D')

In [28]:
from Bio import Entrez
Entrez.email = "nw17@sanger.ac.uk"

handle = Entrez.efetch(db="protein", id="WP_159676815", rettype="gb", retmode="text")
print(handle.readline().strip())

LOCUS       WP_159676815             702 aa            linear   BCT 19-JAN-2020


In [21]:
file = 'blast/' + query + suffix
blast = readblast(file)
blast

# handle = ExPASy.get_prosite_raw('WP_159676815')

,query acc.,subject title,subject acc.,subject tax ids,evalue,bit score,% identity,query length,q. start,q. end,subject length,s. start,s. end,log evalue,q. coverage,s. coverage
0,1_accA1,ATP-grasp domain-containing protein [Enhydroba...,WP_159676815,2653138,0.000000e+00,1330.0,98.433,2109,1,2106,702,1,702,300.000000,0.998103,0.998575
1,1_accA1,acetyl/propionyl/methylcrotonyl-CoA carboxylas...,WP_057758341,1699623,0.000000e+00,892.0,70.000,2109,1,2100,699,1,698,300.000000,0.995258,0.997139
2,1_accA1,ATP-grasp domain-containing protein [Psychroba...,WP_197129152,2792045,0.000000e+00,884.0,68.245,2109,1,2106,709,1,706,300.000000,0.998103,0.994358
3,1_accA1,ATP-grasp domain-containing protein [Psychroba...,WP_201586985,1569264,0.000000e+00,879.0,69.394,2109,1,2094,701,1,696,300.000000,0.992413,0.991441
4,1_accA1,ATP-grasp domain-containing protein [Psychroba...,WP_197099812,2792058,0.000000e+00,875.0,67.922,2109,1,2103,705,1,705,300.000000,0.996681,0.998582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,1_accA1,acetyl/propionyl/methylcrotonyl-CoA carboxylas...,MBH1957596,1891238,0.000000e+00,791.0,62.644,2109,1,2088,663,1,661,300.000000,0.989569,0.995475
397,1_accA1,acetyl/propionyl/methylcrotonyl-CoA carboxylas...,WP_137895302,2502188,0.000000e+00,783.0,64.183,2109,1,2094,664,1,663,300.000000,0.992413,0.996988
398,1_accA1,acetyl/propionyl/methylcrotonyl-CoA carboxylas...,WP_056672315,1736315,0.000000e+00,769.0,62.821,2109,1,2088,670,1,668,300.000000,0.989569,0.995522
399,1_accA1,hypothetical protein FGOOGMKG_03749 [Stenotrop...,VUK11702,40324,1.790000e-45,176.0,31.663,2109,1389,85,539,34,454,44.747147,0.618303,0.779221


In [18]:
from Bio.ExPASy import Prosite

record = Prosite.read(handle)
print(record)

None


In [20]:
# RefSeq to IniProt

import urllib.parse
import urllib.request

url = 'https://www.uniprot.org/uploadlists/'

params = {
'from': 'EMBL',
'to': 'ACC+ID',
'format': 'tab',
'query': 'WP_159676815'
}

data = urllib.parse.urlencode(params)
data = data.encode('utf-8')
req = urllib.request.Request(url, data)
with urllib.request.urlopen(req) as f:
   response = f.read()
print(response.decode('utf-8'))

From	To



In [ ]:
# Plothits

def plothits(starts, ends, qlen, names, colours, nhits):
    fig = go.Figure(go.Scatter(x=[0,0], y=[0,0], mode="markers",marker=dict(color=[80,100], colorscale="YlOrRd",colorbar=dict(title="% identity", x=-0.15, xanchor="left"))), go.Layout(plot_bgcolor="white"))
    fig.add_shape(type="rect", name = 'Query', x0 = 1, x1 = qlen, y0=0.5, y1=1.3, line=dict(color="white"), fillcolor='grey')
    fig.add_annotation(xanchor='left', text='Query', x=qlen, y=(0.5+1.3)/2,font=dict(family="Arial Narrow", size=int(100/nhits), color="#000000"), bgcolor="#ffffff", showarrow=False) # Courier New, monospace
    for i in range(0, starts.shape[0]):
        start = starts[i]
        end = ends[i]
        hit_description = names[i]
        colour = colours[i]
#         fig.add_shape(type="rect", name = hit_description, x0 = start, x1 = end, y0=0.5+i+1, y1=1.3+i+1, line=dict(color="white"), fillcolor=matplotlib.colors.rgb2hex(cm.YlOrRd(colour)[:3]))
        fig.add_shape(type="rect", name = hit_description, x0 = start, x1 = end, y0=0.5+i+1, y1=1.3+i+1, line=dict(color="white"), fillcolor=matplotlib.colors.rgb2hex(cm.YlOrRd(colour/100)))
        fig.add_trace(go.Scatter(text = str(colour), x = [start, end, end, start, start], y=[0.5+i+1, 0.5+i+1, 1.3+i+1, 1.3+i+1, 0.5+i+1], fill="toself", line=dict(color="white")))
        fig.add_annotation(xanchor='left', text=hit_description, x=qlen, y=(0.5+i+1+1.3+i+1)/2,font=dict(family="Arial Narrow", size=int(100/nhits), color="#000000"), bgcolor="#ffffff", showarrow=False) # Courier New, monospace
    fig.update_xaxes(range=[0, qlen*2+3000])
    fig.update_yaxes(range=[nhits+0.5, 0.5], showticklabels=False)
    fig.update_layout(margin=dict(l=10, r=20, t=30, b=0))
    return fig

nhits=20
fig = plothits(blast['q. start'], blast['q. end'], blast['query length'][0], descriptions, range(0,100,5), nhits)
fig.update_layout(showlegend=False, title={'text': 'Swissprot hits and relevant functional annotations', 'y':1, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'})
fig.write_image("figures/" + query + "_sprotfunsocs.png", width=1000, height=60*nhits+60, scale=2)

In [ ]:
query = 'b.anthracis_dnaA'
file = 'blast/' + query + ".nr.blastx.pool"
blast = readblast(file)

blast2 = blast
cutrows = []
lastrow = len(blast['subject tax ids'])
for i in range(0, len(blast['subject tax ids'])):
    if str(blast.loc[i,'subject tax ids']).find(";") != -1:
        taxids = str(blast.loc[i,'subject tax ids']).split(";")
        cutrows.append(i)
        for tax in taxids:
            blast2.loc[lastrow+1,:] = blast.loc[i,:]
            blast2.loc[lastrow+1,'subject tax ids'] = tax
            lastrow = lastrow+1

blast = blast2.drop(cutrows)
blast = blast.reset_index(drop=True)
blast['regulated'] = False

print(blast.shape)

for x in range(0, blast.shape[0]):
    try:
        t = taxoniq.Taxon(blast['subject tax ids'][x])
        for level in t.ranked_lineage:
            if int(level.tax_id) in reg_ids:
                blast.loc[x,'regulated'] = True
            if blast.columns.str.contains(level.rank.name).any():
                blast.loc[x,level.rank.name] = level.scientific_name
            else:
                blast[level.rank.name] = ""
                blast.loc[x,level.rank.name] = level.scientific_name
    except:
        print('Taxon id', blast['subject tax ids'][x], 'is missing')
singletons = blast.species.value_counts().index[blast.species.value_counts()==1]
blast['species_simplified'] = blast['species']
blast.loc[blast.species.isin(singletons), 'species_simplified'] = 'other'



In [ ]:
query = 'b.anthracis_lef'
file = 'blast/' + query + ".nr.blastx.pool"
blast = readblast(file)
print(blast.shape)
print(blast.iloc[300:320,:])


In [57]:
# Donut plots

from utils import *
from plotly.subplots import make_subplots

try:
    reg_ids
except:
    reg_ids = get_reg_ids()


import plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
query = 'b.anthracis_lef'
suffix = 'nr.blastx.pool'

blast = taxdist(query, suffix, reg_ids)
blast = blast.loc[:20,:]

domains = [
    {'x': [0.0, 0.33], 'y': [0.66, 1.0]},
    {'x': [0.33, 0.66], 'y': [0.66, 1.0]},
    {'x': [0.66, 1.0], 'y': [0.66, 1.0]},
    {'x': [0.33, 0.66], 'y': [0.0, 0.33]},
    {'x': [0.66, 1.0], 'y': [0.0, 0.33]},
    {'x': [0.33, 1.0], 'y': [0.33, 1.0]}
]
levels = ['superkingdom', 'phylum', 'class', 'genus', 'species', 'subject title']
sizes = [1,3,6,12,18,25]
# holes = [0,0.3,0.4,0.6,0.8,1]
holes = [0,0.5,0.6,0.7,0.8,0.9]
colour_levels = ['red', 'orange', 'yellow', 'green', 'blue', 'purple']
traces = []

for i in range(len(levels)):
    blast.loc[blast[levels[i]]=='',levels[i]] = blast.loc[blast[levels[i]]=='',levels[i-1]] + "missing"
    blast_collapse = blast.sort_values(levels).groupby(levels[:(i+1)], sort=False)
    domain = domains[i]
    values = list(blast_collapse.count()['bit score'])
    values_scaled = np.asarray(values)/sum(np.asarray(values))*sizes[i]
    print(sum(values_scaled))
    labels = np.concatenate(list(blast_collapse[levels[i]].unique()))
    colours = colourscale(blast['regulated'], blast, levels[i])
    trace = go.Pie(
                hole = holes[i],
                scalegroup = 'one',
                sort = False,
                direction='clockwise',
                values = values_scaled,
                domain = domain,
                labels = labels,
                marker={'colors': colours},
#                 marker={'colors': [colour_levels[i] for j in values_scaled]},
                insidetextorientation='radial'
            )
    traces.append(trace)
    
layout = go.Layout(height = 600,
                   width = 600,
                   autosize = False,
                   title = 'Main title')
fig = go.Figure(data = traces, layout = layout)
fig.write_image("figures/" + query + "." + suffix + "_tax_pie.png", width=700, height=700, scale=2)

print('done')

1.0
2.9999999999999996
6.0
11.999999999999998
18.0
24.999999999999993
done


In [67]:
query = 'b.anthracis_lef'
suffix = 'nr.blastx.pool'
blast = taxdist(query, suffix, reg_ids)

print(blast.shape)

def colourscale(reg_status, counts, averages):
    rmap = cm.get_cmap('OrRd', 100)
    nrmap = cm.get_cmap('Blues', 100)
    colours = []
    for i in range(len(reg_status)):
        # colours.append('rgb' + str((np.array(nrmap(averages[i]/averages.max()))*255)[:3].tolist()).replace('[', '(').replace(']', ')')) if reg_status[i] == 0 else colours.append('rgb' + str((np.array(rmap(averages[i]/averages.max()))*255)[:3].tolist()).replace('[', '(').replace(']', ')'))
        colours.append('rgb' + str((np.array(nrmap(averages[i]/averages.max()))*255)[:3].tolist()).replace('[', '(').replace(']', ')')) if reg_status[i] != counts[i] else colours.append('rgb' + str((np.array(rmap(averages[i]/averages.max()))*255)[:3].tolist()).replace('[', '(').replace(']', ')'))
    return colours

levels = ['superkingdom', 'phylum', 'class', 'genus', 'species', 'subject title']
traces = []
sizes = [1,3.5,9,16,25,34]
holes = [0,0.5,0.62,0.74,0.8,0.85]
for i in range(blast.shape[0]):
    if blast.loc[i,'species'] != "":
        blast.loc[i,'species'] = blast.loc[i,'species'].split(sep=" ")[1]
for i in range(len(levels)): 
    blast.loc[blast[levels[i]]=='',levels[i]] = "missing"
    blast_collapse = blast.sort_values(levels).groupby(levels[:(i+1)], sort=False)
    averages = blast_collapse.mean()['bit score']
    counts = list(blast_collapse.count()['bit score'])
    colours = colourscale(blast_collapse.sum()['regulated'], counts, averages)
    print(len(colours))
    values = list(blast_collapse.count()['bit score'])
    values_scaled = np.asarray(values)/sum(np.asarray(values))*sizes[i]
    print(len(values_scaled))
    labels = np.concatenate(list(blast_collapse[levels[i]].unique()))
    trace = go.Pie(
        hole = holes[i],
        scalegroup = 'one',
        sort = False,
        direction='clockwise',
        values = values_scaled,
        labels = labels,
        marker={'colors': colours}
    )
    traces.append(trace)
fig = go.Figure(data=traces)
fig.update_traces(textposition='inside', textinfo='label')
fig.update_layout(showlegend=False)
fig.write_image("figures/" + query + "." + suffix + "_tax_pie.png", width=700, height=700, scale=2)


(527, 26)
3
3
21
21
32
32
75
75
117
117
249
249


In [65]:
def colourscale(regulated, blast, column):
    levels = ['superkingdom', 'phylum', 'class', 'genus', 'species']
    blast_collapse = blast.sort_values(levels).groupby(column, sort=False)
    averages = blast_collapse.mean()['bit score']
    reg_status = blast_collapse.sum()['regulated']
    counts = list(blast_collapse.count()['bit score'])
    rmap = cm.get_cmap('OrRd', 100)
    nrmap = cm.get_cmap('Blues', 100)
    colours = []
    for i in range(len(reg_status)):
        # colours.append('rgb' + str((np.array(nrmap(averages[i]/averages.max()))*255)[:3].tolist()).replace('[', '(').replace(']', ')')) if reg_status[i] == 0 else colours.append('rgb' + str((np.array(rmap(averages[i]/averages.max()))*255)[:3].tolist()).replace('[', '(').replace(']', ')'))
        colours.append('rgb' + str((np.array(nrmap(averages[i]/averages.max()))*255)[:3].tolist()).replace('[', '(').replace(']', ')')) if reg_status[i] != counts[i] else colours.append('rgb' + str((np.array(rmap(averages[i]/averages.max()))*255)[:3].tolist()).replace('[', '(').replace(']', ')'))
    return colours

# query = 'b.anthracis_lef'
# suffix = 'nr.blastx.pool'

# blast = taxdist(query, suffix, reg_ids)

colours = colourscale(blast['regulated'], blast, 'genus')

def col_pal(colours):
    fig = go.Figure(go.Scatter(x=[0,3], y=[0,3],mode="markers"), go.Layout(plot_bgcolor="white"))
    for i in range(len(colours)):
        start = i
        end = i+0.5
        colour = colours[i]
        fig.add_shape(type="rect", name = hit_description, x0 = start, x1 = end, y0=start, y1=end, line=dict(color="white"), fillcolor=colour)
    fig.update_xaxes(range=[0, len(colours)])
    fig.update_yaxes(range=[0, len(colours)], showticklabels=False)
    fig.write_image("figures/colours.png", width=700, height=700, scale=2)

# fig, ax = plt.subplots()
# for i in range(0, len(colours)):
#     print(colours[i])
# #     print(matplotlib.colors.rgb2hex(colours[i]))
#     ax.fill(i, i, c=cmap(i))               # a blue polygon


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


In [80]:
from process_outputs import *
from utils import *

try:
    reg_ids
except:
    reg_ids = get_reg_ids()

def plot_pie(blast, query, suffix):
    levels = ['superkingdom', 'phylum', 'class', 'genus', 'species', 'subject title']
#     levels = ['superkingdom', 'phylum', 'class', 'genus', 'species']
    colour_levels = ['red', 'orange', 'yellow', 'green', 'blue', 'purple']
    traces = []
    sizes = [1,3.5,9,16,25,34]
    holes = [0,0.5,0.62,0.74,0.8,0.85]
#     sizes = [32,16,8,4,2,1]
#     fig = make_subplots(2,3)
#     for i in range(len(levels)): 
    for i in [0,1,2,3,4,5]: 
            blast.loc[blast[levels[i]]=='',levels[i]] = "missing"
            blast_collapse = blast.sort_values(levels).groupby(levels[:(i+1)], sort=False)
            colours = colourscale(blast['regulated'], blast, levels[i])
            values = list(blast_collapse.count()['bit score'])
            values_scaled = np.asarray(values)/sum(np.asarray(values))*sizes[i]
#             print(values_scaled)
            labels = np.concatenate(list(blast_collapse[levels[i]].unique()))
#             print(1/len(levels)*i, "\t", sum(values), np.asarray(values)/sum(np.asarray(values))*sizes[i])
            print(holes[i], "\t", sum(values), np.asarray(values)/sum(np.asarray(values))*sizes[i])
            trace = go.Pie(
#                 hole = 1/len(levels)*(i*1.2),
                hole = holes[i],
                scalegroup = 'one',
                sort = False,
                direction='clockwise',
                values = values_scaled,
                labels = labels,
                marker={'colors': colours},
#                 marker={'colors': [colour_levels[i] for j in values_scaled]},
                insidetextorientation='radial'
            )
#         )
            traces.append(trace)
#             fig.add_trace(trace)
    fig = go.Figure(data=traces)
    fig.update_traces(textposition='inside', textinfo='label') #
    fig.update_layout(showlegend=False)
    fig.write_image("figures/" + query + "." + suffix + "_tax_pie.png", width=700, height=700, scale=2)
    fig.show()

query = 'VFDB_setA_nt_1015'
suffix = 'nr.blastn'

blast = taxdist(query, suffix, reg_ids)
for i in range(blast.shape[0]):
    blast.loc[i,'species'] = blast.loc[i,'species'].split(sep=" ")[1]

plot_pie(blast, query, suffix)


KeyError: 0

In [95]:
def plot_pie(blast, query, suffix):
    levels = ['superkingdom', 'phylum', 'class', 'genus', 'species']
    traces = []
    sizes = [1,3.5,9,16,25,34]
    holes = [0,0.5,0.62,0.74,0.8,0.85]
#     for i in range(blast.shape[0]):
#         if blast.loc[i,'species'] != "":
#             if len(blast.loc[i,'species'].split(sep=" "))>1:
#                 blast.loc[i,'species'] = blast.loc[i,'species'].split(sep=" ")[1]
    for i in range(len(levels)): 
        blast.loc[blast[levels[i]]=='',levels[i]] = blast.loc[blast[levels[i]]=='',levels[i-1]] + " missing"
        blast_collapse = blast.sort_values(levels).groupby(levels[:(i+1)], sort=False)
        averages = blast_collapse.mean()['bit score']
        counts = list(blast_collapse.count()['bit score'])
        colours = colourscale(blast_collapse.sum()['regulated'], counts, averages)
        values = list(blast_collapse.count()['bit score'])
        values_scaled = np.asarray(values)/sum(np.asarray(values))*sizes[i]
        labels = np.concatenate(list(blast_collapse[levels[i]].unique()))
        trace = go.Pie(
            hole = holes[i],
            scalegroup = 'one',
            sort = False,
            direction='clockwise',
            values = values_scaled,
            labels = labels,
            marker={'colors': colours}
        )
        traces.append(trace)
    fig = go.Figure(data=traces)
    fig.update_traces(textposition='inside', textinfo='label')
    fig.update_layout(showlegend=False)
    fig.write_image("figures/" + query + "." + suffix + "_tax_pie.png", width=700, height=700, scale=2)

query = 'b.anthracis_dnaA'
suffix = 'nr.blastx.pool'

blast = taxdist(query, suffix, reg_ids)
# for i in range(blast.shape[0]):
#     blast.loc[i,'species'] = blast.loc[i,'species'].split(sep=" ")[1]

plot_pie(blast, query, suffix)


Taxon id 2810349 is missing
Taxon id 2809036 is missing
Taxon id 2804502 is missing
Taxon id 2803857 is missing
Taxon id 2796379 is missing
Taxon id 2796378 is missing
Taxon id 2796383 is missing
Taxon id 2796385 is missing
Taxon id 2796388 is missing
Taxon id 2796389 is missing
Taxon id 2796391 is missing
Taxon id 2796395 is missing
Taxon id 2796408 is missing
Taxon id 2796409 is missing
Taxon id 2796414 is missing


In [90]:
blast[blast['species']=='Bacillus anthracis']

,query acc.,subject title,subject acc.,subject tax ids,evalue,bit score,% identity,query length,q. start,q. end,...,regulated,species,genus,family,order,class,phylum,superkingdom,kingdom,species_simplified
140,NC_003997_-_dnaA_gene,chromosomal replication initiator protein DnaA...,PFM18768,1392,0.0,878.0,99.326,1341.0,1.0,1335.0,...,True,Bacillus anthracis,Bacillus,Bacillaceae,Bacillales,Bacilli,Firmicutes,Bacteria,NaN,Bacillus anthracis
147,NC_003997_-_dnaA_gene,chromosomal replication initiator protein DnaA...,WP_098759628,1392,0.0,876.0,98.879,1341.0,1.0,1338.0,...,True,Bacillus anthracis,Bacillus,Bacillaceae,Bacillales,Bacilli,Firmicutes,Bacteria,NaN,Bacillus anthracis
216,NC_003997_-_dnaA_gene,MULTISPECIES: chromosomal replication initiato...,WP_000428025,1392,0.0,880.0,99.327,1341.0,1.0,1338.0,...,True,Bacillus anthracis,Bacillus,Bacillaceae,Bacillales,Bacilli,Firmicutes,Bacteria,NaN,Bacillus anthracis
376,NC_003997_-_dnaA_gene,MULTISPECIES: chromosomal replication initiato...,WP_087959066,1392,0.0,879.0,99.103,1341.0,1.0,1338.0,...,True,Bacillus anthracis,Bacillus,Bacillaceae,Bacillales,Bacilli,Firmicutes,Bacteria,NaN,Bacillus anthracis
471,NC_003997_-_dnaA_gene,MULTISPECIES: chromosomal replication initiato...,WP_000428021,1392,0.0,882.0,99.776,1341.0,1.0,1338.0,...,True,Bacillus anthracis,Bacillus,Bacillaceae,Bacillales,Bacilli,Firmicutes,Bacteria,NaN,Bacillus anthracis
482,NC_003997_-_dnaA_gene,MULTISPECIES: chromosomal replication initiato...,WP_000428021,198094,0.0,882.0,99.776,1341.0,1.0,1338.0,...,True,Bacillus anthracis,Bacillus,Bacillaceae,Bacillales,Bacilli,Firmicutes,Bacteria,NaN,Bacillus anthracis
483,NC_003997_-_dnaA_gene,MULTISPECIES: chromosomal replication initiato...,WP_000428021,205919,0.0,882.0,99.776,1341.0,1.0,1338.0,...,True,Bacillus anthracis,Bacillus,Bacillaceae,Bacillales,Bacilli,Firmicutes,Bacteria,NaN,Bacillus anthracis
484,NC_003997_-_dnaA_gene,MULTISPECIES: chromosomal replication initiato...,WP_000428021,212045,0.0,882.0,99.776,1341.0,1.0,1338.0,...,True,Bacillus anthracis,Bacillus,Bacillaceae,Bacillales,Bacilli,Firmicutes,Bacteria,NaN,Bacillus anthracis
485,NC_003997_-_dnaA_gene,MULTISPECIES: chromosomal replication initiato...,WP_000428021,260799,0.0,882.0,99.776,1341.0,1.0,1338.0,...,True,Bacillus anthracis,Bacillus,Bacillaceae,Bacillales,Bacilli,Firmicutes,Bacteria,NaN,Bacillus anthracis
486,NC_003997_-_dnaA_gene,MULTISPECIES: chromosomal replication initiato...,WP_000428021,261591,0.0,882.0,99.776,1341.0,1.0,1338.0,...,True,Bacillus anthracis,Bacillus,Bacillaceae,Bacillales,Bacilli,Firmicutes,Bacteria,NaN,Bacillus anthracis


In [ ]:
re.sub(' ', '\n', blast.loc[i,'species'])
blast.head()

In [ ]:
# print(record.comments)
# print(record.keywords)
# print(record.cross_references)

import Bio.KEGG.Gene
from Bio.KEGG import REST
from Bio.KEGG import Gene
# from Bio import KEGG
# from KEGG import Gene

request = REST.kegg_get("bcu:BCAH820_0001")
print(request.read())

request = REST.kegg_get("bcu02020")
request.read()

In [ ]:
%matplotlib inline
# import matplotlib.pyplot as plt
# import numpy as np
from PIL import Image
import cv2
import os

def results_page(query):
    imgs = []
    cvs = Image.new('RGB',(3000,1000), (255, 255, 255))
    # taxonomic distribution
    if os.path.isfile('figures/' + query + '.nr.blastn_tax_pie.png'):
        im = Image.open('figures/' + query + '.nr.blastn_tax_pie.png')
        im = im.resize((900,900))
        cvs.paste(im, (0,0))
    else:
        print("No taxonomic distribution image")
    # blacklist
    if os.path.isfile('figures/' + query + '_blacklist.png'):
        im = Image.open('figures/' + query + '_blacklist.png')
        im = im.resize((2000,500))
        cvs.paste(im, (1000,0))
    else:
        print("No blacklist image")
    # functional annotation
    if os.path.isfile('figures/' + query + '_blacklist.png'):
        im = Image.open('figures/' + query + '_sprotfunsocs.png')
        im = im.resize((2000,500))
        cvs.paste(im, (1000,500))
    else:
        print("No annotation image")
    return cvs

query = 'b.anthracis_lef'
img = results_page(query)
img.save("summaries/" + query + ".png")

In [ ]:
https://dash-gallery.plotly.host/Portal/
    https://dash.plotly.com/

In [6]:
# Kraken

import pandas as pd
import taxoniq
import plotly.graph_objects as go

def tax_break(tax_split):
    tax_dict = {}
    position = 1
    qlength = 0
    for i in tax_split:
        [taxid,length] = i.split(':')
        qlength = qlength+int(length)
        if taxid == "A":
            if "A" in tax_dict:
                tax_dict["ambiguous"].append([position, position+int(length)])
            else:
                tax_dict["ambiguous"] = [[position, position+int(length)]]
            next
        elif int(taxid) > 0:
            try:
                t = taxoniq.Taxon(taxid)
                if t.scientific_name in tax_dict:
                    tax_dict[t.scientific_name].append([position, position+int(length)])
                else:
                    tax_dict[t.scientific_name] = [[position, position+int(length)]]
            except:
                print("Missing: ", taxid)
                if "missing from taxoniq" in tax_dict:
                    tax_dict["missing from taxoniq"].append([position, position+int(length)])
                else:
                    tax_dict["missing from taxoniq"] = [[position, position+int(length)]]
        else:
            if "unassigned" in tax_dict:
                tax_dict["unassigned"].append([position, position+int(length)])
            else:
                tax_dict["unassigned"] = [[position, position+int(length)]]
        position = position+int(length)
    return tax_dict, qlength


In [7]:
# visualizing taxonomic breakdown
def plothits(tax_dict, qlength):
    fig = go.Figure(go.Scatter(x=[0,0], y=[0,0],mode="markers",marker=dict(color=[0,100], colorscale="YlOrRd",colorbar=dict(title="% identity", x=-0.15, xanchor="left"))), go.Layout(plot_bgcolor="white"))
    fig.add_shape(type="rect", name = 'Query', x0 = 1, x1 = qlength, y0=0.5, y1=1.3, line=dict(color="white"), fillcolor='grey')
    position = 1
#     for i in range(0, len(taxons)):
    taxcount = 0
    for i in tax_dict:
        for stretch in tax_dict[i]:
            if stretch[1] - stretch[0] < qlength/1000:
                stretch[1] = stretch[1] + qlength/1000
            fig.add_shape(type="rect", x0 = stretch[0], x1 = stretch[1], y0=0.5+taxcount+1, y1=1.3+taxcount+1, fillcolor='black', line=dict(color="white"))
            fig.add_annotation(xanchor='left', text=i, x=1, y=(0.5+taxcount+1+1.3+taxcount+1)/2, bgcolor="#ffffff", showarrow=False)
        taxcount = taxcount + 1
#     fig.update_xaxes(range=[0, sum(lengths)])
    fig.update_layout(margin=dict(l=10, r=20, t=30, b=0))
    return fig


In [9]:
query='microarray_probes'

kraken = pd.read_csv('kraken/' + query + '.fasta_kraken', sep='\t', comment='#', header=None)
for i in range(0, kraken.shape[0]):
    print(kraken.loc[i,1])
#     print("Building tax_dict")
    tax_dict, qlength = tax_break(kraken[4][i].split(' '))
#     print("Building fig")
    fig = plothits(tax_dict, qlength)
    fig.write_image("figures/" + kraken.loc[i,1] + "_kraken.png", width=1800, height=700)
    print("Done")

chr1:109457160-0_B_R_1602772855
Done
chr1:109457233-0_B_F_1602777793
Done
chr1:109457614-0_B_R_1601092847
Done
chr1:109457618-0_B_R_1601092821
Done
chr1:109457943-0_B_F_1602595859
Done
chr1:109458224-0_T_R_1602652468
Done
chr1:109458469-0_T_R_1601082154
Done
chr1:109458772-0_B_R_1602531971
Done
chr1:109459424-0_T_F_1602507568
Done
chr1:109460430-0_B_F_1602535943
Done
chr1:109461426-0_T_R_1602511408
Done
chr1:109464943-0_T_F_1602684394
Done
chr1:109465107-0_B_F_1602648327
Done
chr1:109465202-0_B_F_1602637101
Done
chr1:109465610-0_T_R_1602599600
Done
chr1:109467360-0_B_R_1602773039
Done
chr1:109467594-0_B_R_1601128239
Done
chr1:109467820-0_B_R_1602653913
Done
chr1:109467875-0_T_R_1601128260
Done
chr1:109467926-0_B_F_1601086375
Done
chr1:109468832-0_T_R_1602618538
Done
chr1:109470184-0_B_R_1602493725
Done
chr1:109471266-0_B_F_1602598407
Done
chr1:109474234-0_T_R_1601087510
Done
chr1:109474543-0_T_F_1601128130
Done
chr1:109475576-0_T_R_1601086508
Done
chr1:109475610-0_T_F_1601081690
Done
c

KeyboardInterrupt: 